# Explore CUQIpy Test Problems

This notebook demonstrates the different test problems provided by CUQIpy.


First we import any python packages we need, here simply NumPy to handle array computations:

In [ ]:
import numpy as np

Then we import the functionality we need from CUQIpy:

In [ ]:
from cuqi.distribution import Gaussian, Laplace_diff
from cuqi.problem import BayesianProblem
from cuqi.testproblem import Deconvolution, Deconv_1D, Heat_1D, Poisson_1D, Abel_1D #, Deblur
from cuqi.astra import ParBeamCT_2D

In the following sections of this notebook we break it down and take a slightly closer look at each step. The subsequent notebooks provide additional details.

## 2. Loading a test problem  <a class="anchor" id="TestProblem"></a> 

We specify a small 1D deconvolution test problem:

In [ ]:
# 0  Deconvolution
# 1  Deconv_1D
# 2  Heat_1D
# 3  Poisson_1D
# 4  Abel_1D
# 5  Deconv_2D
# 6  ParBeamCT_2D
# 7  (Deblur)

testprob = 5

if testprob == 0:
    model, data, probInfo = Deconvolution.get_components(dim=50, phantom="Square")
elif testprob == 1:
    model, data, probInfo = Deconv_1D.get_components()
elif testprob == 2:
    model, data, probInfo = Heat_1D.get_components()
elif testprob == 3:
    model, data, probInfo = Poisson_1D.get_components()
elif testprob == 4:
    model, data, probInfo = Abel_1D.get_components(dim=201)
elif testprob == 5:
    model, data, probInfo = ParBeamCT_2D()
    

We take a look at the model and see that it is a CUQIpy LinearModel:

In [ ]:
type(model)

We take a look at the data:

In [ ]:
data

The data is a `CUQIarray`, which is a normal NumPy array further equipped with a few utilities, such as Geometry, which allows us to do plotting conveniently:

In [ ]:
data.plot()

The last thing returned by the test problem was `probInfo` which contains additional information about the test problem, typically it includes the exact solution (phantom) and the exact data. We take a look at both:

In [ ]:
probInfo

In [ ]:
probInfo.exactSolution.plot()

In [ ]:
probInfo.exactData.plot()

## 3. Specifying the Bayesian inverse problem  <a class="anchor" id="Bayesian"></a> 

The deconvolution test problem is a linear problem with additive noise:

$$ \mathbf{b} = \mathbf{A}\mathbf{x}+\mathbf{e},$$
where $\mathbf{A}\in\mathbb{R}^{n\times n}$, $\mathbf{x}, \mathbf{b}\in\mathbb{R}^n$ 
and 
$$
\mathbf{x}\sim\mathcal{N}(\mathbf{0},\sigma_x^2\mathbf{I}_n), \\
\mathbf{e}\sim\mathcal{N}(\mathbf{0},\sigma_e^2\mathbf{I}_n).$$

We need three components to specify a Bayesian problem: The prior, the likelihood (or forward model including the noise model), and the data.

The prior we specify as a simple IID Gaussian:

In [ ]:
std_prior = 0.2
prior = Gaussian(mean=np.zeros(model.domain_dim), std=std_prior)

The noise is also Gaussian and combining with the model we specify this as the likelihood:

In [ ]:
std_noise = 0.05
likelihood = Gaussian(mean=model, std=std_noise)

The prior, likelihood and the observed data are combined in a Bayesian inverse problem:

In [ ]:
IP = BayesianProblem(likelihood, prior, data)

The "completely non-expert approach" to solving (more detailed approaches described in later notebooks) is to simply run the UQ method:

In [ ]:
IP.UQ()

The `UQ` method looks at the components of the inverse problem, chooses a suitable sampler, samples the posterior and presents results visually.

To compare with the exact solution (if available) one can pass it as an input:

In [ ]:
IP.UQ(exact=probInfo.exactSolution)

## 4. Changing the prior  <a class="anchor" id="ChangingPrior"></a> 

It is straightforward to change components of the BayesianProblem. For example if we want to experiment with a different prior we can easily swap it out.

We specify a `Laplace_diff` prior, which is a Laplace distribution on differences between neighbouring function values:

In [ ]:
prior_lap = Laplace_diff(location=np.zeros(n), scale=0.5/n, bc_type='zero')

We update the prior of the inverse problem:

In [ ]:
IP.prior = prior_lap

And rerun the `UQ` method:

In [ ]:
IP.UQ(exact=probInfo.exactSolution)

Note how a different sampler was chosen due to the change of prior, and how the prior has changed the solution to be more similar to the exact solution.

#### Try yourself (optional):  
- Change phantom to another of the options provided by the TestProblem. Hint: use `help(Deconvolution)` to see which phantoms are available.
- Play with the parameters of the Gaussian and Laplace priors to see if a better solution can be obtained.